# Week 03 Assignment Covid

Welcome to **week three** of this course programming 1. You will learn about combining data with pandas and numpy and you will learn to visualize with bokeh. Concretely, you will preprocess the partly Synthetic Covid data in an appropiate format in order to conduct statistical and visual analysis. Learning outcomes:

- Load a tabular dataset
- Inspect the dataset for quality and metadata information
- Combine data from several tables into one dataframe
- Subselect specific data from dataframes
- Reshape the dataset into a format suitable for visual and statistical analysis
- Visualize data using bokeh 
- Use widgets to make the plot interactive (optional)
- Use geomap to plot locations (optional)


Your job is to **visualize the lab values taken for COVID-19 patients of survived versus not survived patients**. 

The assignment consists of 6 parts:

- [part 1: load the data](#0)
     - [Exercise 1.1](#ex-11)
- [part 2: data wrangling](#1)
     - [Exercise 2.1](#ex-21)
- [part 3: more wrangling](#2)
     - [Exercise 3.1](#ex-31)
- [part 4: plot the data](#3)
     - [Exercise 4.1](#ex-41)
- [part 5: plot patient location](#5)
     - [Exercise 5.1](#ex-51)


Part 1 and 4 are mandatory, part 5 is optional (bonus)
To pass the assingnment you need to a score of 60%. 


## About the data

The data is generated by Synthea's COVID-19 module. The data was constructed using three peer-reviewed publications published in the early stages of the global pandemic, when less was known, along with emerging resources, data, publications, and clinical knowledge. The simulation outputs synthetic Electronic Health Records (EHR), including the daily consumption of Personal Protective Equipment (PPE) and other medical devices and supplies. For this assignment the `conditions`, `patients`, `observations`, `careplans` and `encounters` table will be used. The Data is stored in separate tables to avoid redundancy, with as a concequence that tables need to be combined and reorganized in dataframes for analysing purpose.

Source: Walonoski J, Klaus S, Granger E, Hall D, Gregorowicz A, Neyarapally G, Watson A, Eastman J. Synthea™ Novel coronavirus (COVID-19) model and synthetic data set. Intelligence-Based Medicine. 2020 Nov;1:100007. https://doi.org/10.1016/j.ibmed.2020.100007

Please <a href = "https://storage.googleapis.com/synthea-public/10k_synthea_covid19_csv.zip">download</a> the data

#### Covid Patients
Patients are considered Covid patients if they are identified with `CODE` `840539006`


#### Survivors
Patients that had covid and where tested negative after isolation have tested code `94531-1`,  SARS-CoV-2 RNA Pnl Resp NAA+probe (covid-sars test) + a value of `Not detected (qualifier value)`. These patients are considered to be survived covid patients. 

#### Non-Survivors
Patients that did not survived Covid have a `DEATHDATE` which is not null. 


#### Lab values  COVID-19 patients

Patients are monitored for blood and heart conditions once they are admitted in Hospital or under treatment. The lab values of interest are as follow: 

- `48065-7`  Fibrin D-dimer FEU [Mass/volume] in Platelet poor plasma
- `26881-3`   Interleukin 6 [Mass/volume] in Serum or Plasma
- `2276-4` Ferritin [Mass/volume] in Serum or Plasma
- `89579-7` Troponin I.cardiac [Mass/volume] in Serum or Plasma by High sensitivity method
- `731-0` Lymphocytes [#/volume] in Blood by Automated count
- `14804-9` Lactate dehydrogenase [Enzymatic activity/volume] in Serum or Plasma by Lactate to pyruvate reaction


---

<a name='0'></a>
## Part 1: Load the data (20 pt)

Instructions: Load the data of the following files

- conditions.csv
- patients.csv
- observations.csv
- careplans.csv
- encounters.csv

Get yourself familiar with the data. Create some meaningful overviews. Answer the following questions

1. How many patients are there
2. How many covid-patients are there
3. How many patients do have a 'Hospital admission for isolation' encounter
    
<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
    <ul><li>use a unique dataframe for each file, use a meaningful name</li>
    <li>pandas.read_csv() method can be used to read a csv file</li>
    <li>pandas.DataFrame.head() method is often used to inspect the dataframe</li>
    <li>.unique() returns a list of unique values of a column</li>
</ul>
</details>

<a name='ex-11'></a>
### 1.1 Code your solution

In [13]:
import pandas as pd
import numpy as np

#YOUR CODE HERE
df_conditions = pd.read_csv("data\\10k_synthea_covid19_csv\conditions.csv")
df_patients = pd.read_csv("data\\10k_synthea_covid19_csv\patients.csv")
df_observations = pd.read_csv("data\\10k_synthea_covid19_csv\observations.csv")
df_careplans = pd.read_csv("data\\10k_synthea_covid19_csv\careplans.csv")
df_encounters = pd.read_csv("data\\10k_synthea_covid19_csv\encounters.csv")

# df_patients.head()
# df_conditions.head()
# df_encounters.head(5)
# df_observations.head(5)
# df_careplans.head(5)




In [14]:
# df_patients.head()
# df_conditions.head()
# df_encounters.head(5)
# df_observations.head(5)
# df_careplans.head(5)


### 1.2 Test your solution
The following function needs to be called. You can use this as a test. There are however more meaningful overviews 
you can create. 

In [15]:
num_pat = df_patients['Id'].nunique()
num_cov = df_conditions['DESCRIPTION'].value_counts()['COVID-19']
num_admitted = df_encounters['DESCRIPTION'].value_counts()['Hospital admission for isolation (procedure)']
num_died = df_patients['DEATHDATE'].count()

def part1(num_pat, num_cov, num_admitted, num_died):
    print(f'There are {num_pat} patients in total')
    print(f'There are {num_cov} covid patients')
    print(f'There are {num_admitted} admitted patients')
    print(f'{num_died} patients died')

part1(num_pat, num_cov, num_admitted, num_died)


There are 12352 patients in total
There are 8820 covid patients
There are 1867 admitted patients
2352 patients died


### Expected outcome

---

<a name='1'></a>
## Part 2: Data Wrangling: set up the dataframe (30 pt)

In this part we are going to combine data to create a dataframe with values of interest for the lab values analysis. 

We would like a dataframe containing the following information per record (only Covid patients!!!)

- `PATIENT` - the ID of the covid patient
- `days` - the number of days the patient is under observation
- `CODE-Y` - the code of the observation  
- `VALUE` - the lab value of the observation

where only the following observation codes needs to be selected:

- `48065-7`  Fibrin D-dimer FEU [Mass/volume] in Platelet poor plasma
- `26881-3`   Interleukin 6 [Mass/volume] in Serum or Plasma
- `2276-4` Ferritin [Mass/volume] in Serum or Plasma
- `89579-7` Troponin I.cardiac [Mass/volume] in Serum or Plasma by High sensitivity method
- `731-0` Lymphocytes [#/volume] in Blood by Automated count
- `14804-9` Lactate dehydrogenase [Enzymatic activity/volume] in Serum or Plasma by Lactate to pyruvate reaction

The days information is not primarely available and needs to be calculated by substracting observation DATE - START. 

An example of such a dataframe is given below:

In [16]:
#Possible approach:
#Select all the patients with covid from the conditions table
#Combine conditions table (only covid patients) with the patient table into a covid_patient table
#select the only the relevant lab observations from the observations table into a lab_obs table
#merge the covid_patient table with the lab_obs table into a covid_patients_obs table
#clean the covid_patients_obs table (rename columns, select only relevant columns, sort, typecast, add days column)



<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
    <ul><li>you can use pandas.DataFrame.merge() to merge dataframes</li>
    <li>df = df[(df.CODE == condition1 | df.CODE == condition1 )] selects rows with CODE of 2 conditional values</li>
    <li>df.DATE - df.START return days if DATE and START are datetime format</li>
    <li>pd.to_datetime() can be used to typecast to datetime</li>
</ul>
</details>

<a name='ex-21'></a>
### 2.1 Code your solution

In [17]:
## YOUR CODE HERE
## Select all the patients with covid from the conditions table
df_covid_patients = df_conditions.loc[df_conditions['DESCRIPTION'] == 'COVID-19']

## Set start and stop as datetime
df_covid_patients = df_covid_patients.astype({"START": 'datetime64[ns]', "STOP": 'datetime64[ns]'})

## Combine conditions table (only covid patients) with the patient table into a covid_patient table
df_merged = pd.merge(df_covid_patients, df_patients, how = 'inner', left_on = 'PATIENT', right_on = 'Id')

## Codes to select in observation codes in a list:
observation_codes = ['48065-7', '26881-3', '2276-4', '89579-7', '731-0', '14804-9']
# df_observations.head()
###############################################################################################################################################
######## The next line is wrong, returns 73918 instead of 74096 lines.
df_observations = df_observations[(df_observations['CODE'].isin(observation_codes))] #selects rows with code that are in observation_codes
###############################################################################################################################################

## Merge dataframes
df_merged = pd.merge(df_merged, df_observations, how = 'inner', left_on = 'PATIENT', right_on = 'PATIENT')

## Change date to datetime64[ns]
df_merged = df_merged.astype({'DATE' : 'datetime64[ns]'})

## Add days column
df_merged['days'] = df_merged['DATE'] - df_merged['START']

## Select only relevant columns
columns_titles = ['PATIENT','days', 'CODE_y', 'VALUE', 'UNITS', 'LAT', 'LON']
df_merged_covid = df_merged[columns_titles]

df_merged_covid


,PATIENT,days,CODE_y,VALUE,UNITS,LAT,LON
0,f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,0 days,731-0,1.1,10*3/uL,42.740220,-72.722648
1,f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,0 days,48065-7,0.4,ug/mL,42.740220,-72.722648
2,f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,0 days,2276-4,332.4,ug/L,42.740220,-72.722648
3,f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,0 days,89579-7,2.3,pg/mL,42.740220,-72.722648
4,f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,0 days,14804-9,223.9,U/L,42.740220,-72.722648
...,...,...,...,...,...,...,...
73913,c9699449-7a8b-400a-8e86-fab6aa7134cb,8 days,731-0,0.9,10*3/uL,41.749877,-70.198601
73914,c9699449-7a8b-400a-8e86-fab6aa7134cb,8 days,48065-7,0.5,ug/mL,41.749877,-70.198601
73915,c9699449-7a8b-400a-8e86-fab6aa7134cb,8 days,2276-4,525.2,ug/L,41.749877,-70.198601
73916,c9699449-7a8b-400a-8e86-fab6aa7134cb,8 days,89579-7,3.0,pg/mL,41.749877,-70.198601


---

<a name='2'></a>
## Part 3: Data Wrangling, split into survived and not survived (10 pt)

Now we have the required data we would like to split the data into survived and not survived. First we fetch all the ids of the survived and deceased patients. We can use these ids to select the records of the survived patients and the patients that did not survived

Your job is to split the data into survived and not survived records. There are multiple ways to do this. One way is the  `.isin()` method

In [18]:
#the following code is given, RUN THIS CELL
#get survived and deceased ids
completed_isolation_patients = df_careplans[(df_careplans.CODE == 736376001) & (df_careplans.STOP.notna()) \
                                          & (df_careplans.REASONCODE == 840539006)].PATIENT
negative_covid_patient_ids = df_observations[(df_observations.CODE == '94531-1') \
                                          & (df_observations.VALUE == 'Not detected (qualifier value)')].PATIENT.unique()
survivor_ids = np.union1d(completed_isolation_patients, negative_covid_patient_ids)
deceased_ids = df_patients[df_patients.DEATHDATE.notna()].Id

<a name='ex-31'></a>
### 3.1 Code your solution

In [19]:
#YOUR CODE HERE
df_survivors = df_merged_covid.query('PATIENT not in @deceased_ids')
df_deceased = df_merged_covid.query('PATIENT in @deceased_ids')

died = len(df_deceased)
survived = len(df_survivors)


### 3.2 Test your solution

In [20]:
def test3(survived, died):
    print(f'patients records survived: {survived}, patients records deceased {died}')
test3(survived, died)
#call the test3

patients records survived: 57125, patients records deceased 16793


#### Expected outcome

---

<a name='3'></a>
## Part 4: Plot the data (20 pt)

Create plots with the lab data, for each code one plot. Separate the survivors and the deceased by color. An example of such a plot is given below. You can create 6 plots in one grid (for each code one plot) or use a widget (for instance a drop down menu widget) to select a lab CODE. Plot on the x-axis the days, on the y-axis the VALUE. Use proper labels, titles and legends.

<img src="../images/week3_plot.png" width="500" height="500"/>

<a name='ex-41'></a>
### 4.1 Code your solution

In [21]:
#YOUR CODE HERE

from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.io import output_notebook
from bokeh.plotting import ColumnDataSource
from bokeh.models import CustomJS, Dropdown
output_notebook()

Loading BokehJS ...

In [51]:
def plot_measured(code):
    p = figure(plot_width = 500, plot_height = 400, tools='pan, hover')
    p.circle(df_survivors.loc[df_survivors['CODE_y'] == code]['days'],
             df_survivors.loc[df_survivors['CODE_y'] == code]['VALUE'],
             color = 'green', size = 8, alpha=0.5, legend_label='Survived')
    p.circle(df_deceased.loc[df_deceased['CODE_y'] == code]['days'],
             df_deceased.loc[df_deceased['CODE_y'] == code]['VALUE'],
             color = 'red', size = 8, alpha=0.5, legend_label='Deceased')
    p.title.text = 'Covid deceased and survived'
    p.xaxis.axis_label = 'Days'
    p.yaxis.axis_label = df_survivors.loc[df_survivors['CODE_y'] == code]['UNITS'].iloc[1]
    p.legend.location = 'top_left'

    return p

plot_measured('48065-7')

columns = ['48065-7', '26881-3', '2276-4', '89579-7', '731-0', '14804-9']

# dropdown = Dropdown(label="Dropdown button", button_type = "warning", menu=columns)
# dropdown.js_on_event("menu_item_click", CustomJS(code="console.log('dropdown: ' + this.item, this.toString())"))
# dropdown.js_on_event("menu_item_click")


import panel as pn
pn.extension()
# select = pn.widgets.Select(name='Select', options=['Biology', 'Chemistry', 'Physics'])
# select
layout = pn.interact(plot_measured, code=columns)
grid = pn.Row(pn.Column("Choose code", layout[0]), layout[1])
grid


BokehModel(combine_events=True, render_bundle={'docs_json': {'c9079c3b-9148-4853-aa38-7879b51f4312': {'defs': …

Row
    [0] Column
        [0] Markdown(str)
        [1] Column
            [0] Select(name='code', options=['48065-7', '26881-3', ...], value='48065-7')
    [1] Row
        [0] Bokeh(Figure, name='interactive00254')

<a name='4'></a>
## Part 5: Plot the location of the patients (10 pt)

This is a bonus part. Can you plot the patients location on a map? See also 
https://docs.bokeh.org/en/latest/docs/user_guide/geo.html


<a name='ex-51'></a>
### 5.1 Code your solution

In [23]:
import folium
#YOUR CODE HERE

## take dataframes df_survivors and df_deceased and filter for unique patients
df_survivors_unique = df_survivors.drop_duplicates(subset = ['PATIENT'])
df_deceased_unique = df_deceased.drop_duplicates(subset = ['PATIENT'])

## create a map
map_osm = folium.Map(location=[41.8, -71.7], zoom_start = 8)

## .apply(lambda row :: Apply a function along an axis of the DataFrame.
df_survivors_unique.apply(lambda row:folium.Circle(location=[row["LAT"], 
                                                  row["LON"]],color='green').add_to(map_osm), axis=1)
df_deceased_unique.apply(lambda row:folium.Circle(location=[row["LAT"], 
                                                  row["LON"]],color='red').add_to(map_osm), axis=1)

## draw the map
map_osm
